<a href="https://colab.research.google.com/github/salexey1990/gb/blob/master/ml-algorithms/hw08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1
Отобрать наиболее значимые признаки с помощью PCA невозможно, поскольку компоненты, полученные с его помощью более не являются признаками.

# 3

In [0]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import model_selection

In [2]:
# Загрузим игрушечный датасет из sklearn
X, y = datasets.load_iris(return_X_y=True)
X.shape

(150, 4)

In [0]:
# Функция стандартизации признаков
def calc_std_feat(x):
  res = (x - x.mean()) / x.std()
  return res

In [0]:
# Для начала отмасштабируем выборку
X_st = X.copy().astype(np.float64)

X_st[1] = calc_std_feat(X[1])
X_st[2] = calc_std_feat(X[2])
X_st[3] = calc_std_feat(X[3])
X_st[4] = calc_std_feat(X[4])

In [5]:
# Сделаем сингулярное разложение матрицы признаков

u, s, vh = np.linalg.svd(X_st, full_matrices=True)

# В качестве вектора весов возьмём два собственных вектора, соответствующих наибольшим собственным числам

W = vh[:2].T
print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[-0.74981919 -0.2958979 ]
 [-0.37733502 -0.53655644]
 [-0.51643778  0.70371602]
 [-0.16935608  0.35963229]]


In [0]:
# Сформируем новую матрицу "объекты-признаки"
Z = X_st.dot(W)

# 4

Сравним качество предсказания сначала для оригинальных признаков, затем для двух выбранных компонент

In [0]:
# Разобъём данные на обучающую и валидационную выборки

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(X, 
                                                                                     y, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

In [8]:
# Обучим на обучающей выборке модель линейной регрессии и оценим её качество с помощью accuracy метрики

clf = LogisticRegression(random_state=42, multi_class="multinomial", solver="lbfgs").fit(train_data, train_labels)
predictions_1 = clf.predict(test_data)

accuracy_score(test_labels, predictions_1)

0.9777777777777777

In [0]:
# Теперь разобъём на обучающую и валидационную выборку новое признаковое пространство, полученное метом PCA

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(Z, 
                                                                                     y, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

In [11]:
# И проверим качество предсказания по первым двум компонентам

clf = LogisticRegression(random_state=42, multi_class="multinomial", solver="lbfgs").fit(train_data, train_labels)
predictions_2 = clf.predict(test_data)

accuracy_score(test_labels, predictions_2)

0.9777777777777777

Как видим в нашем случае снижение размерности признакового пространства никак ни сказалось на качестве предсказания